50 computers running NIC. Network addresses 0 through 49
Computers are sendig and receiving packets. All packets contain two values named X and Y
Output: destination address, x, y
If the incoming packet queue is empty, provide -1

In [1]:

import utils as u


In [2]:
def parse_code(code):
    A=int(code/10000)
    B=int((code-A*10000)/1000)
    C=int((code-A*10000-B*1000)/100)
    opcode=int((code-A*10000-B*1000-C*100))
    return(A,B,C,opcode)

def read_bc(ic,i,B,C,rb):
    if C==0:
        c=ic[ic[i+1]]
    elif C==1:
        c=ic[i+1]
    else:
        c=ic[ic[i+1]+rb]
    if B==0:
        b=ic[ic[i+2]]
    elif B==1:
        b=ic[i+2]
    else:
        b=ic[ic[i+2]+rb]
    return (b,c)


In [ ]:
class Nic:
    def __init__(self,ic,adress):
        self.i=0
        self.rb=0
        self.ic=ic+[0]*26000
        self.queue=[adress]

    def addqueue(self,inn):
        self.queue=self.queue+inn
    
    def update(self):
        k=0
        output=[]
        while self.i<len(self.ic):
            if self.ic[self.i]==99:
                return output+[99]
            else:
                (A,B,C,opcode) = parse_code(self.ic[self.i])
                (b,c) = read_bc(self.ic,self.i,B,C,self.rb)
                if opcode==1:
                    if A==1:
                        self.ic[self.i+3]=c+b
                    elif A==2:
                        self.ic[self.ic[self.i+3]+self.rb]=c+b
                    else:
                        self.ic[self.ic[self.i+3]]=c+b
                elif opcode==2:
                    if A==1:
                        self.ic[self.i+3]=c*b
                    elif A==2:
                        self.ic[self.ic[self.i+3]+self.rb]=c*b
                    else:
                        self.ic[self.ic[self.i+3]]=c*b
                elif opcode==3:
                    if self.queue==[]:
                        
                        return 'give input'            
                    else:
                        inn=self.queue[0]
                        self.queue=self.queue[1:]
                        if C==1:
                            self.ic[self.i+1]=inn
                        elif C==2:
                            self.ic[self.ic[self.i+1]+self.rb]=inn
                        else:
                            self.ic[self.ic[self.i+1]]=inn
                    
                elif opcode==4:
                    if C==1:
                        output.append(self.ic[self.i+1])
                        #self.i+=2
                        #return output
                    elif C==2:
                        output.append(self.ic[self.ic[self.i+1]+self.rb])
                        #self.i+=2
                        #return output
                    else:
                        output.append(self.ic[self.ic[self.i+1]])
                    #k+=1
                        #self.i+=2
                        #return output
                elif opcode==5:
                    if c!=0:
                        self.i=b
                    else:
                        self.i+=3
                elif opcode==6:
                    if c==0:
                        self.i=b
                    else:
                        self.i+=3
                elif opcode==7:
                    if c<b:
                        if A==1:
                            self.ic[self.i+3]=1
                        elif A==2:
                            self.ic[self.ic[self.i+3]+self.rb]=1
                        else:
                            self.ic[self.ic[self.i+3]]=1
                    else: 
                        if A==1:
                            self.ic[self.i+3]=0
                        elif A==2:
                            self.ic[self.ic[self.i+3]+self.rb]=0
                        else:
                            self.ic[self.ic[self.i+3]]=0
                elif opcode==8:
                    if c==b:
                        if A==1:
                            self.ic[self.i+3]=1
                        elif A==2:
                            self.ic[self.ic[self.i+3]+self.rb]=1
                        else:
                            self.ic[self.ic[self.i+3]]=1
                    else: 
                        if A==1:
                            self.ic[self.i+3]=0
                        elif A==2:
                            self.ic[self.ic[self.i+3]+self.rb]=0
                        else:
                            self.ic[self.ic[self.i+3]]=0
                elif opcode==9:
                    self.rb+=c
                else:
                    return 'error'
            if opcode in (1,2,7,8):
                self.i+=4
            elif opcode in (3,4,9):
                self.i+=2 
            if output[-1]==10:
                return output
            #print(f'opcode:{opcode},i:{i},inn:{inn},output:{output}')
        return 'error'

class NAT:
    def __init__(self):
        self.x=0
        self.y=0
        return None
    
    def update(self):
        return []
    
    def addqueue(self,inn):
        self.x=inn[0]
        self.y=inn[1]
        return None
    
    def idle(self):
        return [self.x,self.y]
        

In [ ]:
ic_org=u.get_input('day23.txt')
ic_org=ic_org.split(',')
ic_org=[int(x) for x in ic_org]

In [ ]:
#startup:
nics={}
for i in range(50):
    nics[i]=Nic(ic_org.copy(),i)
nics[255]=NAT()

In [ ]:
done = False
z=k=0
pipe=[]
while not done:
    for nic in nics:
        out=nics[nic].update()
        if out != []:
            for i in range(int(len(out)/3)):
                nics[out[i*3]].addqueue(out[(i*3)+1:(i+1)*3])
        else:
            k+=1
        if k>200:
            nics[0].addqueue(nics[255].idle())
            pipe.append(nics[255].idle())
            k=0
    if z%1000==0:
        print(z)
    z+=1
    if len(pipe)>2:
        if pipe[-1][1]==pipe[-2][1]:
            done=True
print(pipe[-1][1])

In [ ]:
pipe